In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



## 共通関数定義

In [3]:
params = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
        #,'num_round': 1000
}

## インスタンス生成

In [4]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [5]:
params_base = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'eta': 0.1
        ,'gamma': 0.0
        ,'alpha': 0.0
        ,'lambda': 1.0
        ,'min_child_weight': 6
        ,'max_depth': 1
        ,'subsample': 0.8
        ,'colsample_bytree': 0.8
        ,'random_state': 17
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
        ,'eval_metric': 'auc'
        ,'colsample_bylevel': 0.3
        #,'num_round': 100000
}


def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 10, log=True)
            ,"max_depth": trial.suggest_int("max_depth", 1, 15, step=1, log=False)
            ,"subsample": trial.suggest_float("subsample", 0.1, 0.95,  step =0.05, log=False)
            ,"colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 0.95,  step =0.05, log=False)
            ,'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 0.95,  step =0.05, log=False)
            ,"gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            # 余裕があれば調整する
            ,"alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True)
            ,"lambda": trial.suggest_float("lambda", 1e-6, 10.0, log=True)
            
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = xgb.XGBClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## 特徴量生成

In [6]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 234)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [7]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [8]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [10]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
### 変数選択01
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1168 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1122 features are remained after removing non-informative features
123 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 999 features are remained after removing features not interesting for LightGBM classifier
---=> 58 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1057 features

---=> df final shape: (356250, 1057)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,0,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,1,0,0,0,1,0,1,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,0,0,0,0,0,0


In [13]:
### 変数選択02
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1057 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1051 features are remained after removing non-informative features
40 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1011 features are remained after removing features not interesting for LightGBM classifier
---=> 19 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1030 features

---=> df final shape: (356250, 1030)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [14]:
### 変数選択03
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1030 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1026 features are remained after removing non-informative features
15 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1011 features are remained after removing features not interesting for LightGBM classifier
---=> 19 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1030 features

---=> df final shape: (356250, 1030)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [15]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


## 学習

### 001 baseline

In [16]:
# split train, and test datasets
df = df.replace([np.inf, -np.inf], np.nan)
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1030)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,1,1,0,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,0,1,1,0,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0,1,1,0,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
train_setB1a = pd.read_csv('./home-credit-default-risk/exports/train_setB1a.csv')
test_setB1a =  pd.read_csv('./home-credit-default-risk/exports/test_setB1a.csv')
test_setB1a.head()

,SK_ID_CURR,setB1a
0,100001,0.033433
1,100005,0.098996
2,100013,0.042185
3,100028,0.039429
4,100038,0.161166


In [18]:
app_train = app_train.merge(train_setB1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_setB1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,1,1,0,0,0,0,0,0.033433
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,1,1,0,0,0,0,0,0.098996
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,1,1,0,1,0,1,0,0.042185
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,0,1,1,0,1,0,1,0,0.039429
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,1,0,0,0,0,0,0.161166


In [19]:
train_setB1xgb = pd.read_csv('./home-credit-default-risk/exports/train_setB1xgb.csv')
test_setB1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_setB1xgb.csv')
test_setB1xgb.head()

,SK_ID_CURR,setB1x
0,100001,0.032762
1,100005,0.100620
2,100013,0.044999
3,100028,0.034211
4,100038,0.172532


In [20]:
app_train = app_train.merge(train_setB1xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_setB1xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.033433,0.032762
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,0,0,0,0.098996,0.100620
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,1,0,1,0,0.042185,0.044999
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,1,0,1,0,0.039429,0.034211
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.161166,0.172532


In [21]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [22]:
app_train = app_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.033433,0.032762,0.040068
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.098996,0.100620,0.095240
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.042185,0.044999,0.044435
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.039429,0.034211,0.036586
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.161166,0.172532,0.141754


In [23]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [24]:
app_train = app_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,0,0,0,0.033433,0.032762,0.040068,0.035234
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,0,0,0,0.098996,0.100620,0.095240,0.107426
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,1,0,0.042185,0.044999,0.044435,0.048029
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,1,0,0.039429,0.034211,0.036586,0.032975
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,1,0,0,0,0,0,0.161166,0.172532,0.141754,0.161962


In [25]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [26]:
app_train = app_train.merge(train_set2a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045


In [27]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [28]:
app_train = app_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637


In [29]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.049120
1,100005,0.115729
2,100013,0.064166
3,100028,0.038931
4,100038,0.195575


In [30]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575


In [31]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [32]:
app_train = app_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [33]:
train_set4a = pd.read_csv('./home-credit-default-risk/exports/train_set4a.csv')
test_set4a =  pd.read_csv('./home-credit-default-risk/exports/test_set4a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [34]:
#app_train = app_train.merge(train_set4a, how='left', on='SK_ID_CURR')
#app_test = app_test.merge(test_set4a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [35]:
train_set4xgb = pd.read_csv('./home-credit-default-risk/exports/train_set4xgb.csv')
test_set4xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set4xgb.csv')
test_set4xgb.head()

,SK_ID_CURR,set4x
0,100001,0.034978
1,100005,0.118268
2,100013,0.039774
3,100028,0.037283
4,100038,0.164890


In [36]:
#app_train = app_train.merge(train_set4xgb, how='left', on='SK_ID_CURR')
#app_test = app_test.merge(test_set4xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [37]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [38]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [39]:
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,0,0.283935,0.234856,0.260928,0.199398,0.242499,0.240473,0.223662,0.259405
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,0,0.023060,0.023065,0.012171,0.012061,0.017857,0.014736,0.024137,0.023080
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0.032398,0.038719,0.039950,0.047440,0.033203,0.029793,0.035314,0.033290
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0.028579,0.034008,0.030776,0.030071,0.048900,0.031804,0.031135,0.022860
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0.023252,0.027813,0.035088,0.023555,0.043556,0.060946,0.035167,0.033131


In [40]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [41]:
### 学習モデル構築
train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1036) (61502, 1036)
[11:40:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.79788	validation_1-auc:0.78922
[1]	validation_0-auc:0.80196	validation_1-auc:0.79109
[2]	validation_0-auc:0.80590	validation_1-auc:0.79198
[3]	validation_0-auc:0.80731	validation_1-auc:0.79269
[4]	validation_0-auc:0.80941	validation_1-auc:0.79316
[5]	validation_0-auc:0.81059	validation_1-auc:0.79335
[6]	validation_0-auc:0.81170	validation_1-auc:0.79344
[7]	validation_0-auc:0.81322	validation_1-auc:0.79364
[8]	validation_0-auc:0.81416	validation_1-auc:0.79324
[9]	val

[31]	validation_0-auc:0.83878	validation_1-auc:0.79431
[32]	validation_0-auc:0.84034	validation_1-auc:0.79416
[33]	validation_0-auc:0.84144	validation_1-auc:0.79412
[34]	validation_0-auc:0.84265	validation_1-auc:0.79410
[35]	validation_0-auc:0.84405	validation_1-auc:0.79410
[36]	validation_0-auc:0.84536	validation_1-auc:0.79402
[37]	validation_0-auc:0.84679	validation_1-auc:0.79389
[38]	validation_0-auc:0.84812	validation_1-auc:0.79370
[39]	validation_0-auc:0.84909	validation_1-auc:0.79372
[40]	validation_0-auc:0.85008	validation_1-auc:0.79361
[41]	validation_0-auc:0.85107	validation_1-auc:0.79358
[42]	validation_0-auc:0.85283	validation_1-auc:0.79357
[43]	validation_0-auc:0.85381	validation_1-auc:0.79372
[44]	validation_0-auc:0.85504	validation_1-auc:0.79369
[45]	validation_0-auc:0.85658	validation_1-auc:0.79356
[46]	validation_0-auc:0.85768	validation_1-auc:0.79350
[47]	validation_0-auc:0.85858	validation_1-auc:0.79354
[48]	validation_0-auc:0.85983	validation_1-auc:0.79363
[49]	valid

[71]	validation_0-auc:0.87861	validation_1-auc:0.78774
[72]	validation_0-auc:0.87929	validation_1-auc:0.78776
[73]	validation_0-auc:0.87966	validation_1-auc:0.78769
[74]	validation_0-auc:0.88018	validation_1-auc:0.78758
[75]	validation_0-auc:0.88084	validation_1-auc:0.78768
[76]	validation_0-auc:0.88116	validation_1-auc:0.78769
[77]	validation_0-auc:0.88178	validation_1-auc:0.78762
[78]	validation_0-auc:0.88233	validation_1-auc:0.78766
[79]	validation_0-auc:0.88270	validation_1-auc:0.78770
[80]	validation_0-auc:0.88397	validation_1-auc:0.78770
[81]	validation_0-auc:0.88479	validation_1-auc:0.78760
[82]	validation_0-auc:0.88514	validation_1-auc:0.78758
[83]	validation_0-auc:0.88579	validation_1-auc:0.78750
[84]	validation_0-auc:0.88616	validation_1-auc:0.78753
[85]	validation_0-auc:0.88658	validation_1-auc:0.78749
[86]	validation_0-auc:0.88733	validation_1-auc:0.78740
[87]	validation_0-auc:0.88781	validation_1-auc:0.78732
[88]	validation_0-auc:0.88836	validation_1-auc:0.78727
[89]	valid

[1]	validation_0-auc:0.80106	validation_1-auc:0.79545
[2]	validation_0-auc:0.80362	validation_1-auc:0.79591
[3]	validation_0-auc:0.80529	validation_1-auc:0.79627
[4]	validation_0-auc:0.80695	validation_1-auc:0.79622
[5]	validation_0-auc:0.80857	validation_1-auc:0.79601
[6]	validation_0-auc:0.81046	validation_1-auc:0.79633
[7]	validation_0-auc:0.81160	validation_1-auc:0.79657
[8]	validation_0-auc:0.81250	validation_1-auc:0.79653
[9]	validation_0-auc:0.81403	validation_1-auc:0.79686
[10]	validation_0-auc:0.81512	validation_1-auc:0.79681
[11]	validation_0-auc:0.81611	validation_1-auc:0.79667
[12]	validation_0-auc:0.81720	validation_1-auc:0.79659
[13]	validation_0-auc:0.81804	validation_1-auc:0.79664
[14]	validation_0-auc:0.81891	validation_1-auc:0.79661
[15]	validation_0-auc:0.81967	validation_1-auc:0.79667
[16]	validation_0-auc:0.82077	validation_1-auc:0.79662
[17]	validation_0-auc:0.82226	validation_1-auc:0.79655
[18]	validation_0-auc:0.82370	validation_1-auc:0.79644
[19]	validation_0-a

In [42]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1035,setB1x,0.021670,0.002031
1030,set2a,0.018365,0.001195
1028,set1,0.016807,0.002359
1034,setB1a,0.013172,0.002181
1032,set3a,0.012180,0.001514
1029,set1x,0.007248,0.003785
1033,set3x,0.005701,0.000129
477,EXT_SOURCES_NANMEDIAN,0.002129,0.000271
808,PREV_CODE_REJECT_REASON_CLIENT_MEAN,0.001346,0.000340
146,APP_EXT_SOURCE_2_EXT_SOURCE_3,0.001336,0.000102


In [43]:
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [44]:
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,0,0.283935,0.234856,0.260928,0.199398,0.242499,0.240473,0.223662,0.259405
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,0,0.023060,0.023065,0.012171,0.012061,0.017857,0.014736,0.024137,0.023080
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0.032398,0.038719,0.039950,0.047440,0.033203,0.029793,0.035314,0.033290
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0.028579,0.034008,0.030776,0.030071,0.048900,0.031804,0.031135,0.022860
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0.023252,0.027813,0.035088,0.023555,0.043556,0.060946,0.035167,0.033131


In [45]:
# データセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [46]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [47]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSourceXgb2-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.032829
1,100005,0.103780
2,100013,0.035130
3,100028,0.027752
4,100038,0.154395


### 正規化＋対数変換

In [48]:
app_train_ord = app_train.copy()
app_test_ord = app_test.copy()
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [49]:
# infをnanへ置換
app_train_ord = app_train_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord = app_test_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [50]:
### 欠損値補完
#app_train_ord = app_train_ord.fillna(app_train_ord.max() + (app_train_ord.max() * 0.4))
app_train_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,0,0.283935,0.234856,0.260928,0.199398,0.242499,0.240473,0.223662,0.259405
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,0,0.023060,0.023065,0.012171,0.012061,0.017857,0.014736,0.024137,0.023080
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0.032398,0.038719,0.039950,0.047440,0.033203,0.029793,0.035314,0.033290
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0.028579,0.034008,0.030776,0.030071,0.048900,0.031804,0.031135,0.022860
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0.023252,0.027813,0.035088,0.023555,0.043556,0.060946,0.035167,0.033131


In [51]:
### 欠損値補完
#app_test_ord = app_test_ord.fillna(app_test_ord.max() + (app_test_ord.max() * 0.4))
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [52]:
# infをnanへ置換
app_train_ord = FeatureEngineering2.reduce_mem_usage(app_train_ord)
app_test_ord = FeatureEngineering2.reduce_mem_usage(app_test_ord)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033447,0.032776,0.040070,0.035248,0.042755,0.048981,0.049133,0.034302
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098999,0.100647,0.095215,0.107422,0.103699,0.141357,0.115723,0.127563
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042175,0.045013,0.044434,0.048035,0.036011,0.028091,0.064148,0.044861
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034210,0.036591,0.032990,0.048798,0.039062,0.038940,0.030914
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161133,0.172485,0.141724,0.161987,0.173096,0.192627,0.195557,0.201294


In [53]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(app_test_ord.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [54]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(app_train_ord, app_test_ord, column, [column])

app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033447,0.032776,0.040070,0.035248,0.042755,0.048981,0.049133,0.034302
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098999,0.100647,0.095215,0.107422,0.103699,0.141357,0.115723,0.127563
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042175,0.045013,0.044434,0.048035,0.036011,0.028091,0.064148,0.044861
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034210,0.036591,0.032990,0.048798,0.039062,0.038940,0.030914
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161133,0.172485,0.141724,0.161987,0.173096,0.192627,0.195557,0.201294


In [55]:
### indexを設定
app_train_ord = app_train_ord.set_index('SK_ID_CURR')
app_test_ord = app_test_ord.set_index('SK_ID_CURR')
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0.033447,0.032776,0.040070,0.035248,0.042755,0.048981,0.049133,0.034302
100005,1,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0.098999,0.100647,0.095215,0.107422,0.103699,0.141357,0.115723,0.127563
100013,2,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,0.042175,0.045013,0.044434,0.048035,0.036011,0.028091,0.064148,0.044861
100028,3,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,0.039429,0.034210,0.036591,0.032990,0.048798,0.039062,0.038940,0.030914
100038,4,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0.161133,0.172485,0.141724,0.161987,0.173096,0.192627,0.195557,0.201294


In [56]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train_ord, app_test_ord, num_cols)
        
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,...,-0.498057,0.0,-0.490723,-0.485840,-0.419189,-0.450684,-0.402832,-0.317139,-0.328125,-0.469482
100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,...,-0.498057,0.0,0.188843,0.226562,0.158813,0.319824,0.256592,0.678223,0.364990,0.507324
100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,0.247943,...,2.007802,0.0,-0.400146,-0.357422,-0.373535,-0.314209,-0.475830,-0.541992,-0.171875,-0.358887
100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,2.805819,...,2.007802,0.0,-0.428711,-0.470947,-0.455566,-0.474609,-0.337402,-0.423828,-0.434326,-0.504883
100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,0.235763,...,-0.498057,0.0,0.833496,0.980957,0.646484,0.902344,1.007812,1.230469,1.195312,1.279297


In [57]:
### indexを削除
app_train_ord = app_train_ord.reset_index()
app_test_ord = app_test_ord.reset_index()
app_test_ord.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,...,-0.498057,0.0,-0.490723,-0.485840,-0.419189,-0.450684,-0.402832,-0.317139,-0.328125,-0.469482
1,100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,...,-0.498057,0.0,0.188843,0.226562,0.158813,0.319824,0.256592,0.678223,0.364990,0.507324
2,100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,...,2.007802,0.0,-0.400146,-0.357422,-0.373535,-0.314209,-0.475830,-0.541992,-0.171875,-0.358887
3,100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,...,2.007802,0.0,-0.428711,-0.470947,-0.455566,-0.474609,-0.337402,-0.423828,-0.434326,-0.504883
4,100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,...,-0.498057,0.0,0.833496,0.980957,0.646484,0.902344,1.007812,1.230469,1.195312,1.279297


In [58]:
x_train = app_train_ord.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train_ord['TARGET']
id_train = app_train_ord[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [59]:
# データセット作成
x_test = app_test_ord.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test_ord[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [60]:
#del app_train_ord
#del app_test_ord
#gc.collect()

In [61]:
### 学習モデル構築
train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1036) (61502, 1036)
[11:42:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.79796	validation_1-auc:0.78949
[1]	validation_0-auc:0.80177	validation_1-auc:0.79097
[2]	validation_0-auc:0.80566	validation_1-auc:0.79167
[3]	validation_0-auc:0.80740	validation_1-auc:0.79255
[4]	validation_0-auc:0.80936	validation_1-auc:0.79323
[5]	validation_0-auc:0.81039	validation_1-auc:0.79365
[6]	validation_0-auc:0.81168	validation_1-auc:0.79380
[7]	validation_0-auc:0.81325	validation_1-auc:0.79387
[8]	validation_0-auc:0.81436	validation_1-auc:0.79361
[9]	val

[31]	validation_0-auc:0.83939	validation_1-auc:0.79521
[32]	validation_0-auc:0.84073	validation_1-auc:0.79509
[33]	validation_0-auc:0.84185	validation_1-auc:0.79491
[34]	validation_0-auc:0.84307	validation_1-auc:0.79492
[35]	validation_0-auc:0.84423	validation_1-auc:0.79478
[36]	validation_0-auc:0.84527	validation_1-auc:0.79482
[37]	validation_0-auc:0.84660	validation_1-auc:0.79479
[38]	validation_0-auc:0.84825	validation_1-auc:0.79455
[39]	validation_0-auc:0.84939	validation_1-auc:0.79456
[40]	validation_0-auc:0.85024	validation_1-auc:0.79459
[41]	validation_0-auc:0.85141	validation_1-auc:0.79457
[42]	validation_0-auc:0.85287	validation_1-auc:0.79452
[43]	validation_0-auc:0.85347	validation_1-auc:0.79476
[44]	validation_0-auc:0.85465	validation_1-auc:0.79472
[45]	validation_0-auc:0.85564	validation_1-auc:0.79457
[46]	validation_0-auc:0.85677	validation_1-auc:0.79459
[47]	validation_0-auc:0.85797	validation_1-auc:0.79447
[48]	validation_0-auc:0.85925	validation_1-auc:0.79452
[49]	valid

[70]	validation_0-auc:0.87974	validation_1-auc:0.78859
[71]	validation_0-auc:0.88035	validation_1-auc:0.78851
[72]	validation_0-auc:0.88104	validation_1-auc:0.78849
[73]	validation_0-auc:0.88144	validation_1-auc:0.78844
[74]	validation_0-auc:0.88175	validation_1-auc:0.78845
[75]	validation_0-auc:0.88243	validation_1-auc:0.78855
[76]	validation_0-auc:0.88319	validation_1-auc:0.78837
[77]	validation_0-auc:0.88365	validation_1-auc:0.78840
[78]	validation_0-auc:0.88418	validation_1-auc:0.78828
[79]	validation_0-auc:0.88470	validation_1-auc:0.78832
[80]	validation_0-auc:0.88595	validation_1-auc:0.78822
[81]	validation_0-auc:0.88637	validation_1-auc:0.78820
[82]	validation_0-auc:0.88687	validation_1-auc:0.78815
[83]	validation_0-auc:0.88783	validation_1-auc:0.78826
[84]	validation_0-auc:0.88828	validation_1-auc:0.78820
[85]	validation_0-auc:0.88855	validation_1-auc:0.78818
[86]	validation_0-auc:0.88906	validation_1-auc:0.78807
[87]	validation_0-auc:0.88955	validation_1-auc:0.78801
[88]	valid

[0]	validation_0-auc:0.79683	validation_1-auc:0.79419
[1]	validation_0-auc:0.80099	validation_1-auc:0.79584
[2]	validation_0-auc:0.80319	validation_1-auc:0.79629
[3]	validation_0-auc:0.80520	validation_1-auc:0.79649
[4]	validation_0-auc:0.80705	validation_1-auc:0.79645
[5]	validation_0-auc:0.80884	validation_1-auc:0.79679
[6]	validation_0-auc:0.81076	validation_1-auc:0.79691
[7]	validation_0-auc:0.81217	validation_1-auc:0.79690
[8]	validation_0-auc:0.81276	validation_1-auc:0.79694
[9]	validation_0-auc:0.81413	validation_1-auc:0.79702
[10]	validation_0-auc:0.81498	validation_1-auc:0.79692
[11]	validation_0-auc:0.81600	validation_1-auc:0.79679
[12]	validation_0-auc:0.81724	validation_1-auc:0.79693
[13]	validation_0-auc:0.81812	validation_1-auc:0.79685
[14]	validation_0-auc:0.81896	validation_1-auc:0.79672
[15]	validation_0-auc:0.81986	validation_1-auc:0.79664
[16]	validation_0-auc:0.82109	validation_1-auc:0.79657
[17]	validation_0-auc:0.82259	validation_1-auc:0.79666
[18]	validation_0-au

In [62]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1035,setB1x,0.021915,0.001146
1030,set2a,0.018576,0.000776
1028,set1,0.016530,0.001798
1034,setB1a,0.013173,0.001791
1032,set3a,0.011692,0.001699
1029,set1x,0.008020,0.003605
1033,set3x,0.005226,0.000538
477,EXT_SOURCES_NANMEDIAN,0.001995,0.000415
146,APP_EXT_SOURCE_2_EXT_SOURCE_3,0.001373,0.000144
135,APPS_EXT_SOURCE_MEAN,0.001370,0.000117


In [63]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [64]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSourceXgb2-2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.039521
1,100005,0.112866
2,100013,0.042107
3,100028,0.035372
4,100038,0.159177


## パラメータチューニング

In [65]:
#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [66]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [67]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [68]:
#train_oof, imp, metrics = baseline.train_xgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [69]:
### 推論用データセットの作成
#x_test = app_test.drop(columns=['SK_ID_CURR'])
#id_test = app_test[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_xgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_XgbHyperParameterTuningXgb2-3.csv", index=None)

In [70]:
#del app_train_ord
#del app_test_ord
#gc.collect()